In [10]:
import cv2
import numpy as np
from keras.preprocessing import image
import tensorflow as tf

In [11]:
model = tf.keras.models.load_model('saved_model/my_model')


import json

# Load the train_indices.json file
with open('train_indices.json', 'r') as f:
    train_index = json.load(f)
    


In [12]:

# Create a VideoCapture object to capture video frames
video_capture = cv2.VideoCapture(0)  # Replace '0' with the video file path if using a video file

# Loop to continuously capture and process video frames
while True:
    # Read a frame from the video capture
    ret, frame = video_capture.read()
    
    if not ret:
        break
    
    # Preprocess the frame
    preprocessed_frame = cv2.resize(frame, (64, 64))
    preprocessed_frame = preprocessed_frame / 255.0
    preprocessed_frame = np.expand_dims(preprocessed_frame, axis=0)
    
    # Perform prediction
    predictions = model.predict(preprocessed_frame)
    predicted_index = np.argmax(predictions)
    predicted_label = list(train_index.keys())[list(train_index.values()).index(predicted_index)]
    
    # Print the prediction
    probability = predictions[0][predicted_index]
    print(f"Predicted label: {predicted_label}, Probability: {probability}")
    
    # Display the frame
    cv2.putText(frame, predicted_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('Video', frame)
    
    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close the OpenCV windows
video_capture.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 54ms/step
Predicted label: Jaden, Probability: 0.4892045855522156
1/1 [==============================] - 0s 24ms/step
Predicted label: Jaden, Probability: 0.5050894021987915
1/1 [==============================] - 0s 25ms/step
Predicted label: Jaden, Probability: 0.5082510709762573
1/1 [==============================] - 0s 20ms/step
Predicted label: Jaden, Probability: 0.5082510709762573
1/1 [==============================] - 0s 20ms/step
Predicted label: Jaden, Probability: 0.5159673094749451
1/1 [==============================] - 0s 21ms/step
Predicted label: Jaden, Probability: 0.5058960318565369
1/1 [==============================] - 0s 21ms/step
Predicted label: Jaden, Probability: 0.5058960318565369
1/1 [==============================] - 0s 24ms/step
Predicted label: Jaden, Probability: 0.5087490081787109
1/1 [==============================] - 0s 20ms/step
Predicted label: Jaden, Probability: 0.4884676933288574
1/1 [=======================